In [10]:
! pip install pandas catboost requests beautifulsoup4 seaborn jinja2 selenium matplotlib sklearn tensorflow numpy scikit-learn xgboost lightgbm tensorflow-addons optuna imbalanced-learn


In [3]:
import pandas as pd

# Đọc dữ liệu từ file đã xử lý
df = pd.read_csv(r'E:\vietlot_mega\data_655/vietlott_655_processed.csv')

# Hiển thị số mẫu dữ liệu
print(f'Số lượng mẫu dữ liệu: {len(df)}')

# Nếu bạn muốn xem 5 dòng đầu tiên:
print(df.head())


Số lượng mẫu dữ liệu: 1194
   Ngày Mở Thưởng\tKết Quả\tGiải Jackpot 1\tGiải Jackpot 2
T7   24/05/2025\t19 20 27 30 45 55 15\t146.813.952...     
T5   22/05/2025\t03 09 14 41 47 55 22\t140.519.788...     
T3   20/05/2025\t19 27 44 45 47 52 15\t134.006.693...     
T7   17/05/2025\t02 07 26 29 41 50 43\t124.379.568...     
T5   15/05/2025\t06 09 13 44 49 54 47\t115.812.032...     


In [4]:
import pandas as pd
import json
import os

# Đường dẫn tệp
INPUT_FILE = r"E:\vietlot_mega\data_655\vietlott_655_processed.csv"
OUTPUT_FILE = r"E:\vietlot_mega\data_655\vietlott_655_clean.csv"
DATA_DIR = r"E:\vietlot_mega\data_655"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(DATA_DIR, exist_ok=True)

# Đọc dữ liệu với dấu phân cách tab
try:
    df = pd.read_csv(INPUT_FILE, encoding="utf-8-sig", sep='\t')
except Exception as e:
    raise Exception(f"Lỗi khi đọc file {INPUT_FILE}: {e}")

# In danh sách cột để kiểm tra
print("Các cột trong tệp CSV:", df.columns.tolist())

# Hàm xử lý cột ngày
def process_date(date_str):
    # Loại bỏ tiền tố ngày trong tuần (nếu có, ví dụ: 'T3, ')
    if ", " in str(date_str):
        date_str = date_str.split(", ")[-1]
    return pd.to_datetime(date_str, format="%d/%m/%Y", dayfirst=True, errors='coerce')

# Hàm xử lý cột 'Kết Quả'
def process_result(result_str):
    # Chuyển chuỗi thành danh sách số
    numbers = [int(x) for x in str(result_str).split()]
    # Lấy 6 số chính (bỏ số bổ sung)
    return numbers[:6]

# Xử lý dữ liệu
# Tìm cột ngày (linh hoạt với các tên cột phổ biến)
date_column = None
possible_date_columns = ['Ngày Mở Thưởng', 'Ngay Mo Thuong', 'Date', 'Ngày', 'Draw Date']
for col in possible_date_columns:
    if col in df.columns:
        date_column = col
        break

if date_column is None:
    raise KeyError("Không tìm thấy cột ngày trong CSV. Các cột hiện có: " + str(df.columns.tolist()))

# Kiểm tra cột 'Kết Quả'
if 'Kết Quả' not in df.columns:
    possible_result_columns = ['Ket Qua', 'Result', 'Kết quả']
    for col in possible_result_columns:
        if col in df.columns:
            df = df.rename(columns={col: 'Kết Quả'})
            break
    else:
        raise KeyError("Không tìm thấy cột 'Kết Quả' trong CSV. Các cột hiện có: " + str(df.columns.tolist()))

# Làm sạch tên cột (loại bỏ khoảng trắng thừa)
df.columns = df.columns.str.strip()

# Áp dụng xử lý
df['Ngày'] = df[date_column].apply(process_date)
df['Kết Quả'] = df['Kết Quả'].apply(process_result)

# Kiểm tra giá trị ngày không hợp lệ
invalid_dates = df[df['Ngày'].isna()]
if not invalid_dates.empty:
    raise ValueError(f"Có {len(invalid_dates)} giá trị ngày không hợp lệ:\n" + str(invalid_dates[[date_column, 'Kết Quả']]))

# Kiểm tra tính hợp lệ của kết quả
def validate_result(result):
    try:
        if len(result) != 6:
            return False, "Phải có đúng 6 số chính"
        if not all(isinstance(x, int) and 1 <= x <= 55 for x in result):
            return False, "Số phải trong khoảng 1-55"
        return True, ""
    except:
        return False, "Định dạng không hợp lệ"

df['Valid_Result'] = df['Kết Quả'].apply(validate_result)
invalid_results = df[~df['Valid_Result'].apply(lambda x: x[0])]
if not invalid_results.empty:
    raise ValueError("Dữ liệu chứa kết quả không hợp lệ:\n" + str(invalid_results[['Ngày', 'Kết Quả', 'Valid_Result']]))

# Chuyển danh sách Kết Quả thành chuỗi JSON để lưu
df['Kết Quả'] = df['Kết Quả'].apply(json.dumps)

# Lưu dữ liệu đã xử lý
df[['Ngày', 'Kết Quả']].to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
print(f"Đã lưu dữ liệu đã xử lý vào: {OUTPUT_FILE}")

Các cột trong tệp CSV: ['Ngày Mở Thưởng', 'Kết Quả', 'Giải Jackpot 1', 'Giải Jackpot 2']
Đã lưu dữ liệu đã xử lý vào: E:\vietlot_mega\data_655\vietlott_655_clean.csv


In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Input, Flatten
from tensorflow.keras.callbacks import EarlyStopping, Callback, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import Precision
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import random
from collections import Counter
from datetime import datetime, timedelta
import os
import logging
import warnings
import itertools

warnings.filterwarnings('ignore')

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Thiết lập seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Hằng số cấu hình
FILE_PATH = 'E:\\vietlot_mega\\data_655\\vietlott_655_clean.csv'
SEQUENCE_LENGTH = 30
CONFIG = {
    'num_heads': 4,
    'ff_dim': 128,
    'learning_rate': 0.001,
    'batch_size': 32,
    'epochs': 150,
    'dropout_rate': 0.3,
    'hist_ratio': 0.3
}

# Callback
class PrintTrainingMetrics(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logging.info(f"Epoch {epoch+1}/{self.params['epochs']} - "
                     f"loss: {logs['loss']:.4f} - val_loss: {logs['val_loss']:.4f} - "
                     f"precision_at_k: {logs.get('precision_at_k', 0):.4f} - "
                     f"val_precision_at_k: {logs.get('val_precision_at_k', 0):.4f}")

class VietlottPredictor:
    def __init__(self, sequence_length=SEQUENCE_LENGTH, file_path=FILE_PATH, config=CONFIG):
        self.sequence_length = sequence_length
        self.file_path = file_path
        self.config = config
        self.results = None
        self.df = None
        self.scaler = None
        self.feature_scaler = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.last_sequence = None
        self.statistical_patterns = {}
        self.next_draw = None
        self.next_date = None

    def get_next_draw(self, steps_ahead=1):
        if self.df is None or self.df.empty:
            raise ValueError("Dữ liệu chưa được tải.")
        logging.info("Xác định kỳ quay và ngày quay tiếp theo...")
        self.df['Draw'] = range(1, len(self.df) + 1)
        last_draw = self.df['Draw'].iloc[-1]
        last_date = self.df['Date'].iloc[-1]
        logging.info(f"Ngày cuối trong dữ liệu: {last_date.strftime('%Y-%m-%d')} (Thứ {last_date.weekday()+1})")
        target_days = [1, 3, 5]  # Thứ Ba, thứ Năm, thứ Bảy
        current_date = last_date
        for _ in range(steps_ahead):
            current_day = current_date.weekday()
            days_ahead = min((target - current_day) % 7 or 7 for target in target_days)
            current_date += timedelta(days=days_ahead)
        self.next_date = current_date
        self.next_draw = last_draw + steps_ahead
        logging.info(f"Kỳ quay tiếp theo: {self.next_draw}, Ngày: {self.next_date.strftime('%Y-%m-%d')}")
        return self.next_draw, self.next_date

    def compute_statistical_patterns(self):
        all_numbers = [num for draw in self.results for num in draw]
        freq = Counter(all_numbers)
        total_draws = len(self.results)
        self.statistical_patterns['frequencies'] = {int(k): v for k, v in freq.items()}
        self.statistical_patterns['number_probabilities'] = {int(k): v / (total_draws * 6) for k, v in freq.items()}
        self.statistical_patterns['candidate_numbers'] = sorted(freq, key=freq.get, reverse=True)[:20] or list(range(1, 56))
        recent_numbers = [num for draw in self.results[-15:] for num in draw]
        recent_freq = Counter(recent_numbers)
        self.statistical_patterns['recent_frequencies'] = {int(k): v / 90 for k, v in recent_freq.items()}
        self.statistical_patterns['recent_top_10'] = sorted(recent_freq, key=recent_freq.get, reverse=True)[:10]
        pair_counts = Counter()
        for draw in self.results:
            for pair in itertools.combinations(sorted(draw), 2):
                pair_counts[pair] += 1
        self.statistical_patterns['pair_probabilities'] = {k: v / total_draws for k, v in pair_counts.items()}
        last_appearance = {}
        for i, draw in enumerate(self.results[::-1]):
            for num in draw:
                if num not in last_appearance:
                    last_appearance[num] = i
        self.statistical_patterns['last_appearance'] = last_appearance
        recent_momentum = {}
        recent_draws = self.results[-5:]
        for num in range(1, 56):
            momentum = sum(1 for draw in recent_draws if num in draw) / 5
            recent_momentum[num] = momentum
        self.statistical_patterns['recent_momentum'] = recent_momentum
        logging.debug(f"Candidate numbers: {self.statistical_patterns['candidate_numbers']}")
        logging.debug(f"Recent top 10: {self.statistical_patterns['recent_top_10']}")

    def load_and_preprocess_data(self):
        logging.info("Đang tải và tiền xử lý dữ liệu...")
        try:
            if not os.path.exists(self.file_path):
                logging.error(f"File không tồn tại: {self.file_path}")
                raise FileNotFoundError(f"File không tồn tại: {self.file_path}")
            try:
                self.df = pd.read_csv(self.file_path, encoding='utf-8')
            except UnicodeDecodeError:
                logging.warning("Không thể đọc file với UTF-8, thử encoding latin1...")
                self.df = pd.read_csv(self.file_path, encoding='latin1')
            expected_columns = ['Ngày', 'Kết Quả']
            if not all(col in self.df.columns for col in expected_columns):
                logging.error(f"Thiếu cột cần thiết. Cột có sẵn: {self.df.columns.tolist()}")
                raise ValueError(f"CSV phải chứa các cột: {expected_columns}")
            self.df['Numbers'] = self.df['Kết Quả'].apply(literal_eval)
            self.df = self.df.dropna(subset=['Numbers'])
            self.df['Date'] = pd.to_datetime(self.df['Ngày'], format='%Y-%m-%d', errors='coerce')
            self.df = self.df.dropna(subset=['Date']).sort_values('Date')
            logging.info("5 dòng đầu của dữ liệu:")
            logging.info(f"\n{self.df.head().to_string()}")
            logging.info("5 dòng cuối của dữ liệu:")
            logging.info(f"\n{self.df.tail().to_string()}")
            if len(self.df) < 600:
                logging.error(f"Dữ liệu chỉ có {len(self.df)} kỳ, cần ít nhất 600 kỳ.")
                raise ValueError(f"Dữ liệu chỉ có {len(self.df)} kỳ, cần ít nhất 600 kỳ.")
            if len(set([n for draw in self.df['Numbers'] for n in draw])) < 55:
                logging.error("Dữ liệu không chứa đủ 55 số khác nhau.")
                raise ValueError("Dữ liệu không chứa đủ 55 số khác nhau.")
            if (self.df['Date'].max() - self.df['Date'].min()).days < 1095:
                logging.error("Dữ liệu cần trải dài ít nhất 3 năm.")
                raise ValueError("Dữ liệu cần trải dài ít nhất 3 năm.")
            self.results = self.df['Numbers'].tolist()
            self.compute_statistical_patterns()
            self.df['OddCount'] = self.df['Numbers'].apply(lambda x: sum(1 for n in x if n % 2 == 1))
            self.df['Sum'] = self.df['Numbers'].apply(lambda x: np.log1p(sum(x)))
            self.df['Range'] = self.df['Numbers'].apply(lambda x: np.log1p(max(x) - min(x)))
            self.df['ClusterCount'] = self.df['Numbers'].apply(lambda x: sum(1 for n in x if 1 <= n <= 18) / 6 + sum(1 for n in x if 19 <= n <= 36) / 6 + sum(1 for n in x if 37 <= n <= 55) / 6)
            self.df['RecentFreq'] = self.df['Numbers'].apply(lambda x: sum(self.statistical_patterns['recent_frequencies'].get(n, 0) for n in x))
            self.df['ConsecutiveCount'] = self.df['Numbers'].apply(lambda x: sum(1 for i in range(len(x)-1) if sorted(x)[i+1] == sorted(x)[i] + 1))
            self.df['DeltaSum'] = self.df['Sum'].diff().fillna(0)
            self.df['NumberGaps'] = self.df['Numbers'].apply(lambda x: np.mean([sorted(x)[i+1] - sorted(x)[i] for i in range(len(x)-1)]))
            self.df['RecentHotSpot'] = self.df['Numbers'].apply(lambda x: sum(1 for n in x if n in [num for draw in self.results[-5:] for num in draw]) / 6)
            self.df['PatternScore'] = self.df['Numbers'].apply(lambda x: sum(self.statistical_patterns['pair_probabilities'].get(tuple(sorted([a, b])), 0) for a, b in itertools.combinations(x, 2)))
            self.df['LastAppearance'] = self.df['Numbers'].apply(lambda x: np.mean([self.statistical_patterns['last_appearance'].get(n, len(self.results)) for n in x]))
            self.df['RecentMomentum'] = self.df['Numbers'].apply(lambda x: np.mean([self.statistical_patterns['recent_momentum'].get(n, 0) for n in x]))
            logging.info(f"Dữ liệu đã được tải và tiền xử lý thành công. Số kỳ: {len(self.df)}")
        except Exception as e:
            logging.error(f"Lỗi khi tải dữ liệu: {str(e)}")
            raise

    def prepare_sequences(self):
        logging.info("Chuẩn bị chuỗi dữ liệu...")
        try:
            sequences = []
            targets = []
            features = []
            for i in range(len(self.results) - self.sequence_length):
                seq = self.results[i:i + self.sequence_length]
                target = self.results[i + self.sequence_length]
                seq_features = [
                    self.df['OddCount'].iloc[i:i + self.sequence_length].values,
                    self.df['Sum'].iloc[i:i + self.sequence_length].values,
                    self.df['Range'].iloc[i:i + self.sequence_length].values,
                    self.df['ClusterCount'].iloc[i:i + self.sequence_length].values,
                    self.df['RecentFreq'].iloc[i:i + self.sequence_length].values,
                    self.df['ConsecutiveCount'].iloc[i:i + self.sequence_length].values,
                    self.df['DeltaSum'].iloc[i:i + self.sequence_length].values,
                    self.df['NumberGaps'].iloc[i:i + self.sequence_length].values,
                    self.df['RecentHotSpot'].iloc[i:i + self.sequence_length].values,
                    self.df['PatternScore'].iloc[i:i + self.sequence_length].values,
                    self.df['LastAppearance'].iloc[i:i + self.sequence_length].values,
                    self.df['RecentMomentum'].iloc[i:i + self.sequence_length].values
                ]
                sequences.append(seq)
                targets.append(target)
                features.append(np.stack(seq_features, axis=-1))
            X = np.array(sequences)
            X_features = np.array(features)
            y = np.array(targets)
            self.scaler = MinMaxScaler(feature_range=(0, 1))
            X_reshaped = X.reshape(-1, X.shape[-1])
            X_scaled = self.scaler.fit_transform(X_reshaped)
            X = X_scaled.reshape(X.shape)
            self.feature_scaler = MinMaxScaler(feature_range=(0, 1))
            X_features_reshaped = X_features.reshape(-1, X_features.shape[-1])
            X_features_scaled = self.feature_scaler.fit_transform(X_features_reshaped)
            X_features = X_features_scaled.reshape(X_features.shape)
            X = np.concatenate([X, X_features], axis=-1)
            from sklearn.model_selection import train_test_split
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                X, y, test_size=0.15, random_state=RANDOM_SEED, shuffle=False
            )
            logging.info(f"Kích thước tập huấn luyện: {len(self.X_train)}, tập kiểm tra: {len(self.X_test)}")
            last_sequence = np.array(self.results[-self.sequence_length:])
            last_features = np.stack([
                self.df['OddCount'].iloc[-self.sequence_length:].values,
                self.df['Sum'].iloc[-self.sequence_length:].values,
                self.df['Range'].iloc[-self.sequence_length:].values,
                self.df['ClusterCount'].iloc[-self.sequence_length:].values,
                self.df['RecentFreq'].iloc[-self.sequence_length:].values,
                self.df['ConsecutiveCount'].iloc[-self.sequence_length:].values,
                self.df['DeltaSum'].iloc[-self.sequence_length:].values,
                self.df['NumberGaps'].iloc[-self.sequence_length:].values,
                self.df['RecentHotSpot'].iloc[-self.sequence_length:].values,
                self.df['PatternScore'].iloc[-self.sequence_length:].values,
                self.df['LastAppearance'].iloc[-self.sequence_length:].values,
                self.df['RecentMomentum'].iloc[-self.sequence_length:].values
            ], axis=-1)
            last_sequence_reshaped = last_sequence.reshape(-1, last_sequence.shape[-1])
            last_sequence_scaled = self.scaler.transform(last_sequence_reshaped).reshape(1, self.sequence_length, 6)
            last_features_reshaped = last_features.reshape(-1, last_features.shape[-1])
            last_features_scaled = self.feature_scaler.transform(last_features_reshaped).reshape(1, self.sequence_length, 12)
            self.last_sequence = np.concatenate([last_sequence_scaled, last_features_scaled], axis=-1)
        except Exception as e:
            logging.error(f"Lỗi khi chuẩn bị chuỗi: {str(e)}")
            raise

    def transformer_encoder(self, inputs):
        x = MultiHeadAttention(num_heads=self.config['num_heads'], key_dim=64)(inputs, inputs)
        x = Dropout(self.config['dropout_rate'])(x)
        x = LayerNormalization(epsilon=1e-6)(x + inputs)
        ff = Dense(self.config['ff_dim'], activation='relu')(x)
        ff = Dense(inputs.shape[-1])(ff)
        x = Dropout(self.config['dropout_rate'])(ff)
        x = LayerNormalization(epsilon=1e-6)(x + ff)
        return x

    def build_transformer_model(self):
        inputs = Input(shape=(self.sequence_length, 18))
        x = self.transformer_encoder(inputs)
        x = self.transformer_encoder(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
        x = Dropout(self.config['dropout_rate'])(x)
        outputs = Dense(55, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.config['learning_rate']),
                      loss='binary_crossentropy', metrics=[Precision(top_k=6, name='precision_at_k')])
        return model

    def post_process_numbers(self, nums):
        nums = sorted(list(set(nums)))
        target_len = random.randint(6, 8)  # Chọn ngẫu nhiên 6, 7 hoặc 8 số
        logging.debug(f"Post-process input: {nums}, target length: {target_len}")
        
        # Đảm bảo có ít nhất 2 số hot (19, 27, 45)
        hot_numbers = [19, 27, 45]
        nums = list(set(nums + random.sample(hot_numbers, min(2, len(hot_numbers)))))
        
        # Bổ sung số nếu thiếu
        while len(nums) < target_len:
            available = [n for n in self.statistical_patterns.get('recent_top_10', []) if n not in nums]
            if not available:
                available = [n for n in self.statistical_patterns.get('candidate_numbers', []) if n not in nums]
            if not available:
                available = [n for n in range(1, 56) if n not in nums]
            nums.append(random.choice(available))
            nums = sorted(list(set(nums)))
            logging.debug(f"Supplemented: {nums}")
        
        # Cắt bớt nếu quá dài
        if len(nums) > target_len:
            nums = random.sample(nums, target_len)
            nums = sorted(nums)
            logging.debug(f"Trimmed: {nums}")
        
        max_attempts = 10
        attempt = 0
        while attempt < max_attempts:
            # Kiểm tra tổng
            total = sum(nums)
            if total < 150 or total > 190:
                available = [n for n in self.statistical_patterns.get('recent_top_10', []) if n not in nums]
                if not available:
                    available = [n for n in self.statistical_patterns.get('candidate_numbers', []) if n not in nums]
                if available:
                    idx = random.randint(0, len(nums)-1)
                    nums[idx] = random.choice(available)
                    nums = sorted(list(set(nums)))
                    if len(nums) > target_len:
                        nums = random.sample(nums, target_len)
                        nums = sorted(nums)
            
            # Kiểm tra số lẻ
            odd_count = sum(1 for n in nums if n % 2 == 1)
            if odd_count not in [3, 4]:
                candidates = [n for n in self.statistical_patterns.get('recent_top_10', []) if n % 2 == (1 if odd_count < 3 else 0) and n not in nums]
                if not candidates:
                    candidates = [n for n in self.statistical_patterns.get('candidate_numbers', []) if n % 2 == (1 if odd_count < 3 else 0) and n not in nums]
                if candidates:
                    idx = random.randint(0, len(nums)-1)
                    nums[idx] = random.choice(candidates)
                    nums = sorted(list(set(nums)))
                    if len(nums) > target_len:
                        nums = random.sample(nums, target_len)
                        nums = sorted(nums)
            
            # Kiểm tra phân bố
            if not (any(1 <= n <= 18 for n in nums) and any(19 <= n <= 36 for n in nums) and any(37 <= n <= 55 for n in nums)):
                ranges = [
                    ([n for n in self.statistical_patterns.get('recent_top_10', []) if 1 <= n <= 18], range(1, 19)),
                    ([n for n in self.statistical_patterns.get('recent_top_10', []) if 19 <= n <= 36], range(19, 37)),
                    ([n for n in self.statistical_patterns.get('recent_top_10', []) if 37 <= n <= 55], range(37, 56))
                ]
                for r in ranges:
                    if not any(r[1].start <= n <= r[1].stop - 1 for n in nums):
                        idx = random.randint(0, len(nums)-1)
                        nums[idx] = random.choice(r[0] if r[0] else r[1])
                nums = sorted(list(set(nums)))
                if len(nums) > target_len:
                    nums = random.sample(nums, target_len)
                    nums = sorted(nums)
            
            # Kiểm tra số liên tiếp
            if sum(1 for i in range(len(nums)-1) if nums[i+1] == nums[i] + 1) > 2:
                available = [n for n in self.statistical_patterns.get('recent_top_10', []) if n not in nums]
                if not available:
                    available = [n for n in self.statistical_patterns.get('candidate_numbers', []) if n not in nums]
                if available:
                    idx = random.randint(0, len(nums)-1)
                    nums[idx] = random.choice(available)
                    nums = sorted(list(set(nums)))
                    if len(nums) > target_len:
                        nums = random.sample(nums, target_len)
                        nums = sorted(nums)
            
            # Kiểm tra recent_top_10 (>=4)
            if sum(1 for n in nums if n in self.statistical_patterns.get('recent_top_10', [])) < 4:
                available = [n for n in self.statistical_patterns.get('recent_top_10', []) if n not in nums]
                if available:
                    idx = random.randint(0, len(nums)-1)
                    nums[idx] = random.choice(available)
                    nums = sorted(list(set(nums)))
                    if len(nums) > target_len:
                        nums = random.sample(nums, target_len)
                        nums = sorted(nums)
            
            # Kiểm tra LastAppearance (>=3 in last 5 draws)
            if sum(1 for n in nums if self.statistical_patterns.get('last_appearance', {}).get(n, len(self.results)) <= 5) < 3:
                recent_nums = [n for n in self.statistical_patterns.get('recent_top_10', []) if self.statistical_patterns.get('last_appearance', {}).get(n, len(self.results)) <= 5]
                if recent_nums:
                    idx = random.randint(0, len(nums)-1)
                    nums[idx] = random.choice(recent_nums)
                    nums = sorted(list(set(nums)))
                    if len(nums) > target_len:
                        nums = random.sample(nums, target_len)
                        nums = sorted(nums)
            
            # Điều kiện thoát
            if len(nums) == target_len and 150 <= sum(nums) <= 190 and odd_count in [3, 4] and \
               any(1 <= n <= 18 for n in nums) and any(19 <= n <= 36 for n in nums) and any(37 <= n <= 55 for n in nums) and \
               sum(1 for i in range(len(nums)-1) if nums[i+1] == nums[i] + 1) <= 2 and \
               sum(1 for n in nums if n in self.statistical_patterns.get('recent_top_10', [])) >= 4 and \
               sum(1 for n in nums if self.statistical_patterns.get('last_appearance', {}).get(n, len(self.results)) <= 5) >= 3:
                break
            attempt += 1
        
        # Fallback nếu không thỏa mãn
        if attempt >= max_attempts or not (6 <= len(nums) <= 8):
            logging.info(f"Fallback to random selection with {target_len} numbers: {nums}")
            nums = random.sample(self.statistical_patterns.get('recent_top_10', []) + self.statistical_patterns.get('candidate_numbers', []) + list(range(1, 56)), target_len)
            nums = sorted(nums)
        
        logging.debug(f"Post-processed output: {nums}")
        return nums

    def evaluate_model(self, model, model_type, recent=False):
        try:
            X_eval = self.X_test[-50:] if recent else self.X_test
            y_eval = self.y_test[-50:] if recent else self.y_test
            pred_probs = model.predict(X_eval, verbose=0)
            pred = [np.argsort(p)[-8:][::-1] + 1 for p in pred_probs]  # Lấy tối đa 8 số
            pred = [sorted(list(set(p))) for p in pred]
            for i, p in enumerate(pred):
                logging.debug(f"Trước xử lý (pred {i}): {p}")
                p = self.post_process_numbers(p)
                logging.debug(f"Sau xử lý (pred {i}): {p}")
                if 6 <= len(p) <= 8:
                    pred[i] = p
                else:
                    target_len = random.randint(6, 8)
                    pred[i] = random.sample(range(1, 56), target_len)
                    logging.warning(f"Dự đoán {i} không hợp lệ (len={len(p)}), dùng ngẫu nhiên {target_len} số: {pred[i]}")
            y_true = y_eval.astype(int)
            matches = [len(set(p).intersection(set(t))) for p, t in zip(pred, y_true)]
            match_3 = sum(1 for m in matches if m >= 3) / len(matches) if matches else 0
            match_4 = sum(1 for m in matches if m >= 4) / len(matches) if matches else 0
            logging.info(f"{model_type.capitalize()} {'(Recent)' if recent else ''} - Match 3+: {match_3:.4f}, Match 4+: {match_4:.4f}")
            if match_3 < 0.05:
                logging.warning(f"Match 3+ quá thấp {'(Recent)' if recent else ''}, có thể cần điều chỉnh mô hình hoặc dữ liệu.")
            return match_3
        except Exception as e:
            logging.error(f"Lỗi khi đánh giá {model_type}: {str(e)}")
            raise

    def train_models(self):
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
            ModelCheckpoint('best_transformer_model.h5', monitor='val_precision_at_k', save_best_only=True, mode='max'),
            PrintTrainingMetrics()
        ]
        logging.info("Huấn luyện mô hình Transformer...")
        transformer_model = self.build_transformer_model()
        y_train_bin = np.array([[1 if j+1 in draw else 0 for j in range(55)] for draw in self.y_train], dtype=np.float32)
        y_test_bin = np.array([[1 if j+1 in draw else 0 for j in range(55)] for draw in self.y_test], dtype=np.float32)
        transformer_model.fit(self.X_train, y_train_bin, validation_data=(self.X_test, y_test_bin),
                             epochs=self.config['epochs'], batch_size=self.config['batch_size'], callbacks=callbacks, verbose=0)
        match_3 = self.evaluate_model(transformer_model, 'transformer')
        match_3_recent = self.evaluate_model(transformer_model, 'transformer', recent=True)
        return transformer_model, match_3

    def monte_carlo_simulation(self, probs, n_simulations=120000):
        candidates = []
        for _ in range(n_simulations):
            target_len = random.randint(6, 8)
            nums = np.random.choice(range(1, 56), size=target_len, replace=False, p=probs)
            nums = sorted(list(set(nums)))
            nums = self.post_process_numbers(nums)
            score = sum(self.statistical_patterns.get('number_probabilities', {}).get(n, 0) +
                        self.statistical_patterns.get('recent_frequencies', {}).get(n, 0) +
                        3.0 * self.statistical_patterns.get('recent_momentum', {}).get(n, 0) +
                        sum(self.statistical_patterns.get('pair_probabilities', {}).get(tuple(sorted([n, m])), 0) for m in nums if m > n) -
                        0.1 * self.statistical_patterns.get('last_appearance', {}).get(n, len(self.results))
                        for n in nums)
            candidates.append((nums, score))
        candidates.sort(key=lambda x: x[1], reverse=True)
        return [c[0] for c in candidates[:1]]

    def predict(self, n_predictions=1):
        logging.info(f"Dự đoán {n_predictions} bộ số...")
        try:
            transformer_model, match_3 = self.train_models()
            transformer_probs = transformer_model.predict(self.last_sequence, verbose=0)[0]
            logging.debug(f"Xác suất Transformer: {transformer_probs[:5]}...")
            hist_probs = np.array([self.statistical_patterns.get('number_probabilities', {}).get(i+1, 0) for i in range(55)])
            recent_probs = np.array([self.statistical_patterns.get('recent_frequencies', {}).get(i+1, 0) for i in range(55)])
            random_probs = np.ones(55) / 55
            final_probs = (1 - self.config['hist_ratio']) * transformer_probs + \
                          (self.config['hist_ratio'] * 0.7) * hist_probs + \
                          (self.config['hist_ratio'] * 0.25) * recent_probs + \
                          (self.config['hist_ratio'] * 0.05) * random_probs
            final_probs = final_probs / final_probs.sum() if final_probs.sum() > 0 else np.ones(55) / 55
            predictions = self.monte_carlo_simulation(final_probs, n_simulations=120000)
            df_predictions = pd.DataFrame(predictions, columns=[f'Số {i+1}' for i in range(max(len(p) for p in predictions))])
            df_predictions.to_csv('predictions.csv', index=False)
            logging.info("Đã lưu dự đoán vào predictions.csv")
            return predictions
        except Exception as e:
            logging.error(f"Lỗi khi dự đoán: {str(e)}")
            raise

if __name__ == "__main__":
    try:
        predictor = VietlottPredictor()
        predictor.load_and_preprocess_data()
        next_draw, next_date = predictor.get_next_draw(steps_ahead=1)
        predictor.prepare_sequences()
        predictions = predictor.predict(n_predictions=1)
        print(f"\nDự đoán cho kỳ quay {next_draw} (ngày {next_date.strftime('%Y-%m-%d')}):")
        for i, pred in enumerate(predictions, 1):
            print(f"Bộ {i}: {pred}")
    except Exception as e:
        logging.error(f"Lỗi chính: {str(e)}")

2025-05-29 16:55:47,159 - INFO - Đang tải và tiền xử lý dữ liệu...
2025-05-29 16:55:47,194 - INFO - 5 dòng đầu của dữ liệu:
2025-05-29 16:55:47,198 - INFO - 
            Ngày                   Kết Quả                   Numbers       Date
1193  2017-08-01   [5, 10, 14, 23, 24, 38]   [5, 10, 14, 23, 24, 38] 2017-08-01
1192  2017-08-03    [4, 9, 24, 25, 27, 45]    [4, 9, 24, 25, 27, 45] 2017-08-03
1191  2017-08-05    [1, 5, 11, 32, 40, 45]    [1, 5, 11, 32, 40, 45] 2017-08-05
1190  2017-08-08  [19, 36, 39, 41, 46, 51]  [19, 36, 39, 41, 46, 51] 2017-08-08
1189  2017-08-10  [10, 11, 19, 41, 50, 54]  [10, 11, 19, 41, 50, 54] 2017-08-10
2025-05-29 16:55:47,199 - INFO - 5 dòng cuối của dữ liệu:
2025-05-29 16:55:47,203 - INFO - 
         Ngày                   Kết Quả                   Numbers       Date
4  2025-05-15    [6, 9, 13, 44, 49, 54]    [6, 9, 13, 44, 49, 54] 2025-05-15
3  2025-05-17    [2, 7, 26, 29, 41, 50]    [2, 7, 26, 29, 41, 50] 2025-05-17
2  2025-05-20  [19, 27, 44, 45, 47, 52]